In [ ]:
!pip install tensorflow-datasets scikit-learn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

print("TensorFlow version:", tf.__version__)

In [ ]:
# Download dataset
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

# Load train and test data
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Check data
train_df.head()


In [ ]:
# ---------------------------
# 4. Encode labels (ham=0, spam=1)
# ---------------------------
le = LabelEncoder()
train_df['label_num'] = le.fit_transform(train_df['label'])
test_df['label_num'] = le.transform(test_df['label'])

# Split features and labels
X_train = train_df['message'].values
y_train = train_df['label_num'].values

X_test = test_df['message'].values
y_test = test_df['label_num'].values

In [ ]:
# ---------------------------
# 5. Tokenization and sequence padding
# ---------------------------
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 50
X_train_pad = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

In [ ]:
# ---------------------------
# 6. Build the model
# ---------------------------
model = keras.Sequential([
    layers.Embedding(input_dim=5000, output_dim=16, input_length=max_len),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
# ---------------------------
# 7. Train the model
# ---------------------------
history = model.fit(
    X_train_pad, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test_pad, y_test)
)

In [ ]:
# ---------------------------
# 8. Create prediction function
# ---------------------------
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    pad = keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    prob = model.predict(pad)[0][0]
    label = 'ham' if prob < 0.5 else 'spam'
    return [float(prob), label]

In [ ]:

# ---------------------------
# 9. Test prediction function
# ---------------------------
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print("Prediction:", prediction)

In [ ]:
# ---------------------------
# 10. Run automated test
# ---------------------------
def test_predictions():
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too"
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()
